In [276]:
import pandas as pd
import pyodbc

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [277]:
connection = pyodbc.connect(
  r'Driver={ODBC Driver 17 for SQL Server};'
  r'Server=localhost;'
  r'Database=AdventureWorks2022;'
  r'Trusted_Connection=yes;'
)

In [278]:
#class for reading table
class db_reader:
    def read_table(self, table_name, connection, col=None):
        if col:
            query = f"SELECT {', '.join(col)} FROM {table_name}" #', '.join(col) converts the list of columns into a comma-separated string. For example, if col = ['name', 'salary'], ', '.join(col) produces 'name, salary'.
        else:
            query = f"SELECT * FROM {table_name}"
        return pd.read_sql_query(query, connection)

In [279]:
#class for merging
class merge:
  def merge_tbl(self, t1, t2, join, col = None, col1 = None, col2 = None):
    if col1 and col2:
      return pd.merge(t1, t2, how = join, left_on = col1, right_on = col2)
    elif col:
      return pd.merge(t1, t2, how = join, on = col)

In [280]:
# 101 Retrieve the total number of orders and the average order amount for each customer.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [281]:
soh.groupby('CustomerID').agg({'SalesOrderID' : 'count', 'TotalDue' : 'sum'}).rename(columns={'SalesOrderID' : 'NoOfOrders', 'TotalDue' : 'Total'}).reset_index().head()

,CustomerID,NoOfOrders,Total
0,11000,3,9115.1341
1,11001,3,7054.1875
2,11002,3,8966.0143
3,11003,3,8993.9155
4,11004,3,9056.5911


In [282]:
# 102 Find the number of products in each product category along with the average list price of products within each category.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [283]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [284]:
c = merge().merge_tbl(pro[['ProductID', 'ListPrice', 'ProductSubcategoryID']],sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
c.head()

,ProductID,ListPrice,ProductSubcategoryID,ProductCategoryID
0,680,1431.50,14.0,2
1,706,1431.50,14.0,2
2,707,34.99,31.0,4
3,708,34.99,31.0,4
4,709,9.50,23.0,3


In [285]:
c.groupby('ProductCategoryID').agg({'ProductID' : 'count', 'ListPrice' : 'mean'}).rename(columns = {'ProductID' : 'count', 'ListPrice' : 'avg'}).reset_index()

,ProductCategoryID,count,avg
0,1,97,146.674000
1,2,134,469.860299
2,3,36,49.822222
3,4,29,34.348966


In [286]:
# 103 Retrieve the top 5 customers who have made the highest total purchases.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [287]:
soh.groupby('CustomerID')['TotalDue'].sum().reset_index().sort_values('TotalDue', ascending=False).head()

,CustomerID,TotalDue
18818,29818,989184.0820
18715,29715,961675.8596
18722,29722,954021.9235
18614,29614,924500.0899
19117,30117,919801.8188


In [288]:
# 104 Find the total number of orders and the total sales amount for each month of the year.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [289]:
soh.groupby([soh['OrderDate'].dt.year.rename('Year'), soh['OrderDate'].dt.month.rename('Month')]).agg({'SalesOrderID' : 'count', 'TotalDue' : 'sum'}).reset_index().head()

,Year,Month,SalesOrderID,TotalDue
0,2011,5,43,5.670209e+05
1,2011,6,141,5.070965e+05
2,2011,7,231,2.292183e+06
3,2011,8,250,2.800576e+06
4,2011,9,157,5.547916e+05


In [290]:
# 105 Retrieve the customer who has made the highest single purchase.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [291]:
soh.sort_values('TotalDue', ascending=False).head(1)

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
7471,51131,8,2013-05-30,2013-06-11,2013-06-06,5,False,SO51131,PO16298169856,10-4020-000599,29641,281.0,4,1088,1088,5,4830.0,27578Vi25371,NaN,163930.3943,17948.5186,5608.9121,187487.825,None,45E8CF5F-1D87-4282-8EAA-1903927F84EF,2013-06-06


In [292]:
# 106 Find the top 3 best-selling products based on the total quantity sold.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [293]:
sod.groupby('ProductID')['OrderQty'].sum().reset_index().sort_values('OrderQty', ascending=False).head(3)

,ProductID,OrderQty
5,712,8309
139,870,6815
4,711,6739


In [294]:
# 107 Retrieve the average number of days it took to ship orders for each customer.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [295]:
soh['avg_Diff_Of_Days'] = (soh['ShipDate'] - soh['OrderDate']).dt.days.mean()

In [296]:
soh[['SalesOrderID', 'OrderDate', 'ShipDate', 'avg_Diff_Of_Days']].head()

,SalesOrderID,OrderDate,ShipDate,avg_Diff_Of_Days
0,43660,2011-05-20,2011-06-07,7.000636
1,43661,2011-05-31,2011-06-07,7.000636
2,43662,2011-05-31,2011-06-07,7.000636
3,43663,2011-05-31,2011-06-07,7.000636
4,43664,2011-05-31,2011-06-07,7.000636


In [297]:
# 108 Find the number of orders placed by each customer in the year 2023 along with their total purchase amount

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [298]:
soh_2014 = soh[soh['OrderDate'].dt.year == 2014]

In [299]:
soh_2014.groupby(['CustomerID', soh_2014['OrderDate'].dt.year]).agg({'SalesOrderID' : 'count', 'TotalDue' : 'sum'}).reset_index().head()

,CustomerID,OrderDate,SalesOrderID,TotalDue
0,11001,2014,1,650.8008
1,11012,2014,1,6.9394
2,11013,2014,1,82.8529
3,11017,2014,1,820.2968
4,11018,2014,1,874.4086


In [300]:
# 109 Retrieve the total sales amount for each product category

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [301]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [302]:
c = merge().merge_tbl(pro[['ProductID', 'ProductSubcategoryID']],sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ProductCategoryID
0,680,14.0,2
1,706,14.0,2
2,707,31.0,4
3,708,31.0,4
4,709,23.0,3


In [303]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [304]:
s = merge().merge_tbl(c,sod[['SalesOrderID', 'LineTotal', 'ProductID']], 'inner', 'ProductID')
s.head()

,ProductID,ProductSubcategoryID,ProductCategoryID,SalesOrderID,LineTotal
0,707,31.0,4,43665,20.1865
1,707,31.0,4,43668,40.3730
2,707,31.0,4,43673,80.7460
3,707,31.0,4,43677,20.1865
4,707,31.0,4,43678,20.1865


In [305]:
s.groupby('ProductCategoryID')['LineTotal'].sum().reset_index()

,ProductCategoryID,LineTotal
0,1,9.463085e+07
1,2,1.180259e+07
2,3,2.120383e+06
3,4,1.271992e+06


In [306]:
# 110 Find the number of orders placed on each day of the week along with the average order amount.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [307]:
soh.groupby(soh['OrderDate'].dt.day_name()).agg({'SalesOrderID' : 'count', 'TotalDue' : 'mean'}).reset_index()

,OrderDate,SalesOrderID,TotalDue
0,Friday,4245,3041.581583
1,Monday,4875,4371.979524
2,Saturday,4483,4200.449918
3,Sunday,4444,3753.954094
4,Thursday,4346,3771.836634
5,Tuesday,4481,3620.735702
6,Wednesday,4591,4544.056730


In [308]:
# 111 Retrieve the top 3 sales representatives based on the total sales amount generated from their orders.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [309]:
soh.groupby('SalesPersonID')['TotalDue'].sum().reset_index().sort_values('TotalDue', ascending= False).head(3)

,SalesPersonID,TotalDue
2,276.0,1.169502e+07
3,277.0,1.134239e+07
1,275.0,1.047537e+07


In [310]:
# 112 Find the average quantity of products sold per order

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [311]:
sod.groupby('SalesOrderID')['OrderQty'].mean().reset_index().head()

,SalesOrderID,OrderQty
0,43660,1.000000
1,43661,2.533333
2,43662,2.454545
3,43663,1.000000
4,43664,1.750000


In [312]:
# 113 Retrieve the top 5 customers who have the highest average order amount.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [313]:
soh.groupby('CustomerID')['TotalDue'].mean().reset_index().sort_values('TotalDue', ascending= False).head()

,CustomerID,TotalDue
18641,29641,151704.902175
18923,29923,123454.220225
18624,29624,121734.398425
18861,29861,115043.714775
18940,29940,110784.146350


In [314]:
# 114 Find the number of orders placed by each customer along with their total purchase amount and rank them within each category based on the total purchase amount.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [315]:
a = soh.groupby('CustomerID').agg({'TotalDue': 'sum', 'SalesOrderID' : 'count'}).reset_index().sort_values('TotalDue', ascending=False)
a.head()

,CustomerID,TotalDue,SalesOrderID
18818,29818,989184.0820,12
18715,29715,961675.8596,12
18722,29722,954021.9235,12
18614,29614,924500.0899,13
19117,30117,919801.8188,12


In [316]:
a['Rank']=a['TotalDue'].rank(ascending=False, method='dense').head()

In [317]:
a.head()

,CustomerID,TotalDue,SalesOrderID,Rank
18818,29818,989184.0820,12,1.0
18715,29715,961675.8596,12,2.0
18722,29722,954021.9235,12,3.0
18614,29614,924500.0899,13,4.0
19117,30117,919801.8188,12,5.0


In [318]:
# 115 Retrieve the product category with the highest total sales amount.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [319]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [320]:
c = merge().merge_tbl(pro[['ProductID', 'ProductSubcategoryID']],sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ProductCategoryID
0,680,14.0,2
1,706,14.0,2
2,707,31.0,4
3,708,31.0,4
4,709,23.0,3


In [321]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [322]:
s = merge().merge_tbl(c,sod[['SalesOrderID', 'LineTotal', 'ProductID']], 'inner', 'ProductID')
s.head()

,ProductID,ProductSubcategoryID,ProductCategoryID,SalesOrderID,LineTotal
0,707,31.0,4,43665,20.1865
1,707,31.0,4,43668,40.3730
2,707,31.0,4,43673,80.7460
3,707,31.0,4,43677,20.1865
4,707,31.0,4,43678,20.1865


In [323]:
s.groupby('ProductCategoryID')['LineTotal'].sum().reset_index()

,ProductCategoryID,LineTotal
0,1,9.463085e+07
1,2,1.180259e+07
2,3,2.120383e+06
3,4,1.271992e+06


In [324]:
# 116 Find the median order amount for each month.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [325]:
soh.groupby(soh['OrderDate'].dt.month_name())['TotalDue'].median().reset_index()

,OrderDate,TotalDue
0,April,866.83940
1,August,615.03750
2,December,820.29680
3,February,772.50360
4,January,905.50330
5,July,635.35290
6,June,654.45840
7,March,913.71900
8,May,1105.48340
9,November,865.20400


In [326]:
# 117 Retrieve the top 3 product categories with the highest average sales amount per order

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [327]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [328]:
c = merge().merge_tbl(pro[['ProductID', 'ProductSubcategoryID']],sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ProductCategoryID
0,680,14.0,2
1,706,14.0,2
2,707,31.0,4
3,708,31.0,4
4,709,23.0,3


In [329]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [330]:
s = merge().merge_tbl(c,sod[['SalesOrderID', 'LineTotal', 'ProductID']], 'inner', 'ProductID')
s.head()

,ProductID,ProductSubcategoryID,ProductCategoryID,SalesOrderID,LineTotal
0,707,31.0,4,43665,20.1865
1,707,31.0,4,43668,40.3730
2,707,31.0,4,43673,80.7460
3,707,31.0,4,43677,20.1865
4,707,31.0,4,43678,20.1865


In [331]:
s.groupby('ProductCategoryID')['LineTotal'].mean().reset_index().sort_values('LineTotal', ascending=False).head(3)

,ProductCategoryID,LineTotal
0,1,2364.352583
1,2,631.222232
2,3,99.129621


In [332]:
# 118 Find the customers who have made at least 5 orders and have spent more than $5000 in total, and list them in descending order of their total purchase amount.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [333]:
soh.groupby('CustomerID').agg({'SalesOrderID' : 'count', 'TotalDue' : 'sum'}).reset_index().query('SalesOrderID > 5 & TotalDue > 5000').head()

,CustomerID,SalesOrderID,TotalDue
241,11241,7,12673.4532
242,11242,7,12230.1493
417,11417,7,12429.5465
420,11420,7,12376.8506
425,11425,6,11634.1343


In [334]:
# 119 Retrieve the top 5 sales representatives who have the highest average sales amount per order.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [335]:
soh.groupby('SalesPersonID')['TotalDue'].mean().reset_index().sort_values('TotalDue', ascending=False).head()

,SalesPersonID,TotalDue
6,280.0,39455.222335
7,281.0,29998.214364
16,290.0,29074.155497
2,276.0,27978.514499
15,289.0,27543.462493


In [336]:
# 120 Find the number of orders placed in each quarter of the year along with the total sales amount for each quarter

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [337]:
soh.groupby([soh['OrderDate'].dt.quarter.rename('Quarter'), soh['OrderDate'].dt.year.rename('Year')])['TotalDue'].sum().reset_index().head()

,Quarter,Year,TotalDue
0,1,2012,9.443737e+06
1,1,2013,8.771886e+06
2,1,2014,1.437328e+07
3,2,2011,1.074117e+06
4,2,2012,9.935495e+06


In [338]:
# 121 Retrieve the order date in the format 'YYYY-MM-DD' for all orders.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [339]:
soh['OrderDate'] = pd.to_datetime(soh['OrderDate'])

# Format OrderDate to 'YYYY-MM-DD'
soh['FormattedOrderDate'] = soh['OrderDate'].dt.strftime('%Y-%m-%d')

In [340]:
soh[['SalesOrderID', 'FormattedOrderDate']].head()

,SalesOrderID,FormattedOrderDate
0,43660,2011-05-20
1,43661,2011-05-31
2,43662,2011-05-31
3,43663,2011-05-31
4,43664,2011-05-31


In [341]:
# 122 Convert the list price of products to integer values.

pro =db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [342]:
pro['converted'] = pro['ListPrice'].astype(int)

In [343]:
pro[['Name', 'converted']].head()

,Name,converted
0,Adjustable Race,1000
1,Bearing Ball,0
2,BB Ball Bearing,0
3,Headset Ball Bearings,0
4,Blade,0


In [344]:
# 123 Retrieve the order quantity as floating-point numbers for all orders.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [345]:
sod['converted'] = sod['OrderQty'].astype(float)

In [346]:
sod[['SalesOrderID', 'converted']].head()

,SalesOrderID,converted
0,43660,1.0
1,43660,1.0
2,43661,1.0
3,43661,1.0
4,43661,2.0


In [347]:
# 124 Convert the order date to the day of the year (1-365) for all orders

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [348]:
soh['Date'] = soh['OrderDate'].dt.dayofyear

In [349]:
soh[['SalesOrderID', 'OrderDate', 'Date']].head()

,SalesOrderID,OrderDate,Date
0,43660,2011-05-20,140
1,43661,2011-05-31,151
2,43662,2011-05-31,151
3,43663,2011-05-31,151
4,43664,2011-05-31,151


In [350]:
# 125 Retrieve the last modification date of products in the format 'DD-MM-YYYY HH:MI:SS'.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [351]:
soh['Converted'] = soh['ModifiedDate'].dt.strftime('%d-%m-%Y %h:%m:%s')

In [352]:
soh[['SalesOrderID', 'Converted']].head()

,SalesOrderID,Converted
0,43660,2011-06-07 00:00:00
1,43661,2011-06-07 00:00:00
2,43662,2011-06-07 00:00:00
3,43663,2011-06-07 00:00:00
4,43664,2011-06-07 00:00:00


In [353]:
# 126 Convert the order status code to its corresponding description for all orders.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [354]:
status_map = {
  1 : 'In process',
  2 : 'Approved',
  3 : 'Backordered',
  4 : 'Rejected',
  5 : 'Shipped',
  6 : 'Cancelled'
}

In [355]:
soh['Status'] = soh['Status'].map(status_map)

In [356]:
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,Shipped,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,Shipped,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,Shipped,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,Shipped,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,Shipped,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [357]:
# 127 Retrieve the ship date as a date without time for all orders.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [358]:
soh['OrderDate'] = soh['OrderDate'].dt.date

In [359]:
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [360]:
# 128 Convert the birth date of employees to their corresponding age

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

In [361]:
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [362]:
emp['age'] = pd.Timestamp.now().year - pd.to_datetime(emp['BirthDate']).dt.year

In [363]:
emp[['BusinessEntityID', 'age']].head()

,BusinessEntityID,age
0,1,55
1,2,53
2,3,50
3,4,50
4,5,72


In [364]:
# 129 Retrieve the total due amount of orders as currency formatted strings.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [365]:
soh['TotalDue'] = soh['TotalDue'].map('₹{:,.3f}'.format)

In [366]:
soh[['SalesOrderID', 'TotalDue']].head()

,SalesOrderID,TotalDue
0,43660,"₹1,457.329"
1,43661,"₹36,865.801"
2,43662,"₹32,474.932"
3,43663,₹472.311
4,43664,"₹27,510.411"


In [367]:
# 130 Convert the product weight from pounds to kilograms.

pro = db_reader().read_table('production.product', connection)
pro.tail()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,146.674,44,CM,LB,19.77,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08 10:01:36.827
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,146.674,48,CM,LB,20.13,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08 10:01:36.827
503,999,"Road-750 Black, 52",BK-R19B-52,True,True,Black,100,75,343.6496,146.674,52,CM,LB,20.42,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,AE638923-2B67-4679-B90E-ABBAB17DCA31,2014-02-08 10:01:36.827
504,1001,Vintage Handcrafted Wooden Handlebars,VHWH1000,True,True,GOLD,3,4,6.9223,8.900,M,CM,LB,1.50,1,M,H,U,23.0,18.0,2011-05-31 10:00:00,2011-08-31 10:00:00,2012-05-31 10:00:00,4FBD47F2-B7E5-48B0-928E-45B8933A56DA,2014-02-08 10:02:36.827
505,2006,BIKE HANDLE,RV-H123,True,True,BLUE,250,20,145.8800,2000.000,20,CM,LB,2.00,0,None,None,None,NaN,NaN,2008-04-30 00:00:00,NaT,NaT,D67538F2-A81A-4A4F-98F7-202002D888B4,2024-03-02 14:57:55.763


In [368]:
pro['Weight'] = pro['Weight'] * 0.453592

In [369]:
pro.tail()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,146.674,44,CM,LB,8.967514,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08 10:01:36.827
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,146.674,48,CM,LB,9.130807,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08 10:01:36.827
503,999,"Road-750 Black, 52",BK-R19B-52,True,True,Black,100,75,343.6496,146.674,52,CM,LB,9.262349,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,AE638923-2B67-4679-B90E-ABBAB17DCA31,2014-02-08 10:01:36.827
504,1001,Vintage Handcrafted Wooden Handlebars,VHWH1000,True,True,GOLD,3,4,6.9223,8.900,M,CM,LB,0.680388,1,M,H,U,23.0,18.0,2011-05-31 10:00:00,2011-08-31 10:00:00,2012-05-31 10:00:00,4FBD47F2-B7E5-48B0-928E-45B8933A56DA,2014-02-08 10:02:36.827
505,2006,BIKE HANDLE,RV-H123,True,True,BLUE,250,20,145.8800,2000.000,20,CM,LB,0.907184,0,None,None,None,NaN,NaN,2008-04-30 00:00:00,NaT,NaT,D67538F2-A81A-4A4F-98F7-202002D888B4,2024-03-02 14:57:55.763


In [370]:
# 131 Retrieve the order date and ship date of orders, converting them to the format 'YYYY-MM-DD HH:MM:SS'.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [371]:
soh['OrderDate'] = soh['OrderDate'].dt.strftime('%d-%m-%Y %h:%m:%s')

In [372]:
soh['ShipDate'] = soh['ShipDate'].dt.strftime('%d-%m-%Y %h:%m:%s')

In [373]:
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20 00:00:00,2011-06-12,2011-06-07 00:00:00,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31 00:00:00,2011-06-12,2011-06-07 00:00:00,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31 00:00:00,2011-06-12,2011-06-07 00:00:00,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31 00:00:00,2011-06-12,2011-06-07 00:00:00,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31 00:00:00,2011-06-12,2011-06-07 00:00:00,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [374]:
# 132 List all products whose list prices are converted to euros (assuming 1 USD = 0.84 EUR), rounded to two decimal places.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [375]:
pro['ListPrice'] = pro['ListPrice'] * 0.84

In [376]:
pro['ListPrice'] = pro['ListPrice'].map('${:,.2f}'.format)

In [377]:
pro[['ProductID', 'ListPrice']].head()

,ProductID,ListPrice
0,1,$840.00
1,2,$0.00
2,3,$0.00
3,4,$0.00
4,316,$0.00


In [378]:
# 133 Retrieve the total sales amount of each order, converting it to a monetary format with commas and two decimal places.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [379]:
soh['MonetaryFormat'] = soh['TotalDue'].map('${:,.2f}'.format)

In [380]:
soh[['SalesOrderID', 'TotalDue', 'MonetaryFormat']].head()

,SalesOrderID,TotalDue,MonetaryFormat
0,43660,1457.3288,"$1,457.33"
1,43661,36865.8012,"$36,865.80"
2,43662,32474.9324,"$32,474.93"
3,43663,472.3108,$472.31
4,43664,27510.4109,"$27,510.41"


In [381]:
# 134 Find the age of employees, converting their birth dates to years.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [382]:
emp['age'] = pd.Timestamp.now().year - pd.to_datetime(emp['BirthDate']).dt.year

In [383]:
emp['Year'] = pd.to_datetime(emp['BirthDate']).dt.year

In [384]:
emp[['BusinessEntityID','Year','age']].head()

,BusinessEntityID,Year,age
0,1,1969,55
1,2,1971,53
2,3,1974,50
3,4,1974,50
4,5,1952,72


In [385]:
# 135 Retrieve the date and time of each order, converting them to the format 'DayOfWeek, Month Day, Year HH:MM AM/PM'.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [386]:
soh['Converted'] = soh['OrderDate'].dt.strftime('%A, %B %d, %Y %I:%M %p')

In [387]:
# %A: Full weekday name (e.g., 'Monday')
# %B: Full month name (e.g., 'January')
# %d: Day of the month as a zero-padded decimal number (e.g., '01' to '31')
# %Y: Year with century as a decimal number (e.g., '0001', '0002', ..., '2013', '2014', ..., '9998', '9999')
# %I: Hour (12-hour clock) as a zero-padded decimal number (e.g., '01' to '12')
# %M: Minute as a zero-padded decimal number (e.g., '00' to '59')
# %p: AM or PM designation

In [388]:
soh[['SalesOrderID', 'Converted']].head()

,SalesOrderID,Converted
0,43660,"Friday, May 20, 2011 12:00 AM"
1,43661,"Tuesday, May 31, 2011 12:00 AM"
2,43662,"Tuesday, May 31, 2011 12:00 AM"
3,43663,"Tuesday, May 31, 2011 12:00 AM"
4,43664,"Tuesday, May 31, 2011 12:00 AM"


In [389]:
# 136 List all products whose weights are converted to kilograms (assuming 1 pound = 0.45359237 kilograms), rounded to two decimal places.

pro = db_reader().read_table('production.product', connection)
pro.head()


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [390]:
pro['Weight'] = pro['Weight']*0.45359237

In [391]:
pro['Weight'] = pro['Weight'].map('{:,.2f} kg'.format)

In [392]:
pro.tail()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,146.674,44,CM,LB,8.97 kg,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08 10:01:36.827
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,146.674,48,CM,LB,9.13 kg,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08 10:01:36.827
503,999,"Road-750 Black, 52",BK-R19B-52,True,True,Black,100,75,343.6496,146.674,52,CM,LB,9.26 kg,4,R,L,U,2.0,31.0,2013-05-30 00:00:00,NaT,NaT,AE638923-2B67-4679-B90E-ABBAB17DCA31,2014-02-08 10:01:36.827
504,1001,Vintage Handcrafted Wooden Handlebars,VHWH1000,True,True,GOLD,3,4,6.9223,8.900,M,CM,LB,0.68 kg,1,M,H,U,23.0,18.0,2011-05-31 10:00:00,2011-08-31 10:00:00,2012-05-31 10:00:00,4FBD47F2-B7E5-48B0-928E-45B8933A56DA,2014-02-08 10:02:36.827
505,2006,BIKE HANDLE,RV-H123,True,True,BLUE,250,20,145.8800,2000.000,20,CM,LB,0.91 kg,0,None,None,None,NaN,NaN,2008-04-30 00:00:00,NaT,NaT,D67538F2-A81A-4A4F-98F7-202002D888B4,2024-03-02 14:57:55.763


In [393]:
# 137 Retrieve the total sales amount of each order, converting it to scientific notation.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [394]:
soh['TotalDue'] = soh['TotalDue'].map('{:,.2e}'.format)

In [395]:
soh[['SalesOrderID', 'TotalDue']].head()

,SalesOrderID,TotalDue
0,43660,1.46e+03
1,43661,3.69e+04
2,43662,3.25e+04
3,43663,4.72e+02
4,43664,2.75e+04


In [396]:
# 138 Find the length of product names, converting the lengths to binary format.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [397]:
pro['len'] = pro['Name'].str.len()
pro['len'].head()

0    15
1    12
2    15
3    21
4     5
Name: len, dtype: int64

In [398]:
pro[['Name', 'len']].head()

,Name,len
0,Adjustable Race,15
1,Bearing Ball,12
2,BB Ball Bearing,15
3,Headset Ball Bearings,21
4,Blade,5


In [399]:
pro['len'] = pro['len'].apply(lambda x: format(x, 'b'))

In [400]:
pro[['Name', 'len']].head()

,Name,len
0,Adjustable Race,1111
1,Bearing Ball,1100
2,BB Ball Bearing,1111
3,Headset Ball Bearings,10101
4,Blade,101


In [401]:
# 139 Retrieve the hire date of employees, converting them to the format 'Day, Month DD, YYYY'.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [402]:
emp['HireDate'] = pd.to_datetime(emp['HireDate'])

In [403]:
emp['HireDate'] = emp['HireDate'].dt.strftime('%A, %B %d, %Y')

In [404]:
emp[['BusinessEntityID', 'HireDate']].head()

,BusinessEntityID,HireDate
0,1,"Wednesday, January 14, 2009"
1,2,"Thursday, January 31, 2008"
2,3,"Sunday, November 11, 2007"
3,4,"Wednesday, December 05, 2007"
4,5,"Sunday, January 06, 2008"


In [405]:
# 140 List all products whose dimensions (height, width, depth) are converted to cubic centimeters


In [406]:
# 141 Retrieve the orders where the total amount is greater than $1000 and less than $2000.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [407]:
soh['TotalDue'] = (soh['TotalDue'] > 1000) & (soh['TotalDue'] < 2000)

In [408]:
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,True,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,False,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,False,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,False,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,False,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [409]:
# 142 List all products that are either in category 'Bikes' or have a list price greater than $1000

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [410]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [411]:
c = merge().merge_tbl(pro[['ProductID', 'ProductSubcategoryID', 'ListPrice']], sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner','ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [412]:
cat = db_reader().read_table('production.productcategory', connection)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [413]:
n = merge().merge_tbl(c, cat[['ProductCategoryID', 'Name']], 'inner', 'ProductCategoryID')
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
0,680,14.0,1431.50,2,Components
1,706,14.0,1431.50,2,Components
2,707,31.0,34.99,4,Accessories
3,708,31.0,34.99,4,Accessories
4,709,23.0,9.50,3,Clothing


In [414]:
n = n[(n['Name'] == 'Bikes') | (n['ListPrice'] > 1000)]

In [415]:
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
0,680,14.0,1431.5,2,Components
1,706,14.0,1431.5,2,Components
12,717,14.0,1431.5,2,Components
13,718,14.0,1431.5,2,Components
14,719,14.0,1431.5,2,Components


In [416]:
# 143 Retrieve the orders where the ship date is NULL or the difference between the order date and ship date is greater than 7 days.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [417]:
soh['diff'] = (pd.to_datetime(soh['ShipDate']) - pd.to_datetime(soh['OrderDate'])).dt.days
soh['diff']

0        18
1         7
2         7
3         7
4         7
5         7
6         7
7         7
8         7
9         7
10        7
11        7
12        7
13        7
14        7
15        7
16        7
17        7
18        7
19        7
20        7
21        7
22        7
23        7
24        7
25        7
26        7
27        7
28        7
29        7
30        7
31        7
32        7
33        7
34        7
35        7
36        7
37        7
38        7
39        7
40        7
41        7
42        7
43        7
44        7
45        7
46        7
47        7
48        7
49        7
50        7
51        7
52        7
53        7
54        7
55        7
56        7
57        7
58        7
59        7
60        7
61        7
62        7
63        7
64        7
65        7
66        7
67        7
68        7
69        7
70        7
71        7
72        7
73        7
74        7
75        7
76        7
77        7
78        7
79        7
80        7
81        7
82        7
83  

In [418]:
soh = soh[(soh['ShipDate'].isnull()) | (soh['diff'] > 7)]
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate,diff
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07,18
21429,65089,9,2014-01-28,2014-02-10,2014-02-05,5,False,SO65089,PO986112457,10-4020-000518,30079,288.0,8,743,743,5,206.0,46635Vi1057,11077.0,1466.0100,125.6580,39.2681,1630.9361,None,91858E70-0B81-4E96-8CD1-A25A7A79EA3A,2014-02-05,8
21430,65090,9,2014-01-28,2014-02-10,2014-02-05,5,False,SO65090,PO5481126157,10-4020-000014,29812,288.0,8,715,715,5,6291.0,88255Vi32672,11077.0,98.7740,7.9019,2.4694,109.1453,None,9363F316-FD43-4003-8B8F-1D220DC39083,2014-02-05,8
23542,67202,9,2014-02-28,2014-03-13,2014-03-08,5,False,SO67202,PO1682155237,10-4020-000572,29745,288.0,8,746,746,5,8579.0,17820Vi44368,NaN,249.5428,19.9634,6.2386,275.7448,None,B52687C1-C0A9-4063-96C2-4789E03227B6,2014-03-08,8
23543,67203,9,2014-02-28,2014-03-13,2014-03-08,5,False,SO67203,PO5423131670,10-4020-000639,29837,287.0,8,750,750,5,11888.0,58269Vi61420,11418.0,2465.2298,214.5225,67.0383,2746.7906,None,C6856A42-A289-4BF3-ADEA-86BE8F1E8DE8,2014-03-08,8


In [419]:
# 144 Find the employees who have either 'Sales Representative' or 'Sales Manager' as their job title.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [420]:
emp = emp[(emp['JobTitle'] == 'Sales Representative') | (emp['JobTitle'] == 'Sales Manager')]
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
274,275,841560125,adventure-works\michael9,3.0,Sales Representative,1968-12-25,S,M,2011-05-31,True,38,39,True,E19F8DE2-0F40-418C-84A2-DB48B98DC6A2,2014-06-30
275,276,191644724,adventure-works\linda3,3.0,Sales Representative,1980-02-27,M,F,2011-05-31,True,27,33,True,4509F387-D73A-43DC-A502-B1C27AA1DC9E,2014-06-30
276,277,615389812,adventure-works\jillian0,3.0,Sales Representative,1962-08-29,S,F,2011-05-31,True,24,32,True,A011C97B-081C-4739-996F-C1CAC4532F8D,2014-06-30
277,278,234474252,adventure-works\garrett1,3.0,Sales Representative,1975-02-04,M,M,2011-05-31,True,33,36,True,50EECC16-0D0D-43A9-9649-016C06DE8D78,2014-06-30
278,279,716374314,adventure-works\tsvi0,3.0,Sales Representative,1974-01-18,M,M,2011-05-31,True,29,34,True,BB510FCE-0105-4306-B591-6450D9EBF401,2014-06-30


In [421]:
# 145 Retrieve the orders where the customer has a credit limit greater than $5000 and the payment due is not yet received.

In [422]:
# 146 List all products with a list price less than $500 but not in the 'Accessories' category.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [423]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [424]:
c = merge().merge_tbl(pro[['ProductID', 'ProductSubcategoryID', 'ListPrice']], sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner','ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [425]:
cat = db_reader().read_table('production.productcategory', connection)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [426]:
n = merge().merge_tbl(c, cat[['ProductCategoryID', 'Name']], 'inner', 'ProductCategoryID')
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
0,680,14.0,1431.50,2,Components
1,706,14.0,1431.50,2,Components
2,707,31.0,34.99,4,Accessories
3,708,31.0,34.99,4,Accessories
4,709,23.0,9.50,3,Clothing


In [427]:
n = n[(n['Name'] != 'Accessories') & (n['ListPrice'] < 500)]

In [428]:
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
4,709,23.0,9.50,3,Clothing
5,710,23.0,9.50,3,Clothing
7,712,19.0,8.99,3,Clothing
8,713,21.0,49.99,3,Clothing
9,714,21.0,49.99,3,Clothing


In [429]:
# 147 Retrieve the orders where the payment due date is before the ship date or the payment due amount is greater than the total due amount.


In [430]:
# 148 Find the employees who started working after January 1, 2015, and are not in the 'Production Technician' role

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [431]:
emp['HireDate'] = pd.to_datetime(emp['HireDate'])

In [432]:
emp = emp[(emp['HireDate'] > '2015-01-01') & (emp['JobTitle'] != 'Production Technician')] 
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate


In [433]:
# 149 Retrieve the orders where the payment due date is not in the future and the payment status is not 'Paid'.


In [434]:
# 150 List all customers whose email address is either null or contains 'gmail.com'.

email = db_reader().read_table('person.emailaddress', connection)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [435]:
email = email[(email['EmailAddress'].isnull()) | email['EmailAddress'].str.contains('gmail.com')]
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07


In [436]:
# 151 Retrieve the orders where the total amount is greater than $1000 or the payment status is 'Partial'.

In [437]:
# 152 List all products that are either out of stock or have a list price greater than $2000

In [438]:
# 153 Retrieve the orders where the ship date is NULL and the payment status is not 'Paid'.

In [439]:
# 154 Find the employees who have either 'Sales Representative' or 'Sales Manager' as their job title and have a salary greater than $50000.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [440]:
emp = emp[(emp['JobTitle'] != 'Sales Representative') | (emp['JobTitle'] != 'Sales Manager')] 
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [441]:
# 155 Retrieve the orders where the payment due date is before the ship date or the payment status is 'Overdue'.

In [442]:
# 156 List all products with a list price less than $500 but not in the 'Bikes' category.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [443]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [444]:
c = merge().merge_tbl(pro[['ProductID', 'ProductSubcategoryID', 'ListPrice']], sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner','ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [445]:
cat = db_reader().read_table('production.productcategory', connection)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [446]:
n = merge().merge_tbl(c, cat[['ProductCategoryID', 'Name']], 'inner', 'ProductCategoryID')
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
0,680,14.0,1431.50,2,Components
1,706,14.0,1431.50,2,Components
2,707,31.0,34.99,4,Accessories
3,708,31.0,34.99,4,Accessories
4,709,23.0,9.50,3,Clothing


In [447]:
n = n[(n['Name'] != 'Bikes') & (n['ListPrice'] < 500)]

In [448]:
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
2,707,31.0,34.99,4,Accessories
3,708,31.0,34.99,4,Accessories
4,709,23.0,9.50,3,Clothing
5,710,23.0,9.50,3,Clothing
6,711,31.0,34.99,4,Accessories


In [449]:
# 157 Retrieve the orders where the payment due date is not in the future or the payment status is 'Pending'.

In [450]:
# 158 Find the employees who started working after January 1, 2015, and have a job title other than 'Sales Manager'.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [451]:
emp['HireDate'] = pd.to_datetime(emp['HireDate'])

In [452]:
emp = emp[(emp['HireDate'] > '2015-01-01') & (emp['JobTitle'] != 'Sales Manager')] 
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate


In [453]:
# 159 Retrieve the orders where the payment due date is not in the past and the payment status is not 'Paid' or 'Partial'.

In [454]:
# 160 List all customers whose email address is either null or contains 'hotmail.com' but not 'gmail.com'.

email = db_reader().read_table('person.emailaddress', connection)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [455]:
email = email[(email['EmailAddress'].isnull()) | email['EmailAddress'].str.contains('gmail.com') | email['EmailAddress'].str.contains('hotmail.com')]
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
19972,20778,19973,aravndh@hotmail.com,052B671A-1FAB-4FE2-B45C-035882DFCF34,2009-01-22


In [456]:
# 162 Retrieve the orders where the total amount is greater than $5000 and the payment status is either 'Pending' or 'Partial'.

In [457]:
# 163 List all products that are either in stock or have a list price greater than $1500 and belong to the 'Road Bikes' category.

In [458]:
# 164 Retrieve the orders where the ship date is NULL and the payment status is 'Pending', or the payment due amount is greater than $500.

In [459]:
# 165 Find the employees who have 'Senior' in their job title or have a salary greater than $80000.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [460]:
emp = emp[emp['JobTitle'].str.contains('Senior')] 
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
10,11,974026903,adventure-works\ovidiu0,3.0,Senior Tool Designer,1978-01-17,S,M,2010-12-05,False,7,23,True,F68C7C19-FAC1-438C-9BB7-AC33FCC341C3,2014-06-30
13,14,42487730,adventure-works\michael8,3.0,Senior Design Engineer,1979-06-16,S,M,2010-12-30,True,3,21,True,46286CA4-46DD-4DDB-9128-85B67E98D1A9,2014-06-30


In [461]:
# 166 Retrieve the orders where the payment due date is before the ship date and the payment status is 'Overdue', or the total amount is greater than $10000.

In [462]:
# 167 List all products with a list price less than $1000 but not in the 'Components' category.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [463]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [464]:
c = merge().merge_tbl(pro[['ProductID', 'ProductSubcategoryID', 'ListPrice']], sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner','ProductSubcategoryID')
c.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID
0,680,14.0,1431.50,2
1,706,14.0,1431.50,2
2,707,31.0,34.99,4
3,708,31.0,34.99,4
4,709,23.0,9.50,3


In [465]:
cat = db_reader().read_table('production.productcategory', connection)
cat.head()

,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30


In [466]:
n = merge().merge_tbl(c, cat[['ProductCategoryID', 'Name']], 'inner', 'ProductCategoryID')
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
0,680,14.0,1431.50,2,Components
1,706,14.0,1431.50,2,Components
2,707,31.0,34.99,4,Accessories
3,708,31.0,34.99,4,Accessories
4,709,23.0,9.50,3,Clothing


In [467]:
n = n[(n['Name'] != 'Components') & (n['ListPrice'] < 1000)]

In [468]:
n.head()

,ProductID,ProductSubcategoryID,ListPrice,ProductCategoryID,Name
2,707,31.0,34.99,4,Accessories
3,708,31.0,34.99,4,Accessories
4,709,23.0,9.50,3,Clothing
5,710,23.0,9.50,3,Clothing
6,711,31.0,34.99,4,Accessories


In [469]:
# 168 Retrieve the orders where the payment due date is in the future or the payment status is either 'Pending' or 'Partial'.


In [470]:
# 169 Find the employees who started working after January 1, 2018, and are not in the 'Marketing' department

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [471]:
emp['HireDate'] = pd.to_datetime(emp['HireDate'])

In [472]:
emp = emp[(emp['HireDate'] > '2015-01-01') & (emp['JobTitle'] != 'Sales Manager')] 
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate


In [473]:
# 170 Retrieve the orders where the payment due date is not in the past and the payment status is not 'Paid' and not 'Pending'.

In [474]:
# 171 List all customers whose email address is either null or contains 'yahoo.com' and does not contain 'gmail.com'.

email = db_reader().read_table('person.emailaddress', connection)
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07
1,2,2,terri0@adventure-works.com,B5FF9EFD-72A2-4F87-830B-F338FDD4D162,2008-01-24
2,3,3,roberto0@adventure-works.com,C8A51084-1C03-4C58-A8B3-55854AE7C499,2007-11-04
3,4,4,rob0@adventure-works.com,17703ED1-0031-4B4A-AFD2-77487A556B3B,2007-11-28
4,5,5,gail0@adventure-works.com,E76D2EA3-08E5-409C-BBE2-5DD1CDF89A3B,2007-12-30


In [475]:
email = email[(email['EmailAddress'].isnull()) | email['EmailAddress'].str.contains('gmail.com') | email['EmailAddress'].str.contains('yahoo.com')]
email.head()

,BusinessEntityID,EmailAddressID,EmailAddress,rowguid,ModifiedDate
0,1,1,newmail@gmail.com,8A1901E4-671B-431A-871C-EADB2942E9EE,2009-01-07


In [476]:
# 172 Retrieve the sales order ID, order date, and the running total sales amount for each order, sorted by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [477]:
soh['RunningTotal'] = soh['TotalDue'].cumsum()

In [478]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue','RunningTotal']].head()

,SalesOrderID,OrderDate,TotalDue,RunningTotal
0,43660,2011-05-20,1457.3288,1457.3288
1,43661,2011-05-31,36865.8012,38323.1300
2,43662,2011-05-31,32474.9324,70798.0624
3,43663,2011-05-31,472.3108,71270.3732
4,43664,2011-05-31,27510.4109,98780.7841


In [479]:
# 173 Find the product ID, name, and the cumulative sum of quantities sold for each product, ordered by product ID.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [480]:
sod['CumulativeSum'] = sod.groupby('ProductID')['OrderQty'].cumsum()

In [481]:
sod[['ProductID', 'OrderQty', 'CumulativeSum']].sort_values(['ProductID', 'CumulativeSum']).head(10)

,ProductID,OrderQty,CumulativeSum
50,707,1,1
74,707,2,3
126,707,4,7
151,707,1,8
172,707,1,9
179,707,4,13
194,707,1,14
227,707,3,17
308,707,4,21
313,707,2,23


In [482]:
# 174 Retrieve the employee ID, hire date, and the running count of employees hired over time, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [580]:
emp['Year'] = pd.to_datetime(emp['HireDate']).dt.year

In [583]:
emp['RunningTotal'] = emp.groupby('Year')['BusinessEntityID'].cumcount()

In [584]:
emp[['BusinessEntityID', 'HireDate', 'RunningTotal']].sort_values('HireDate').head(10)

,BusinessEntityID,HireDate,RunningTotal
27,28,2006-06-30,0
16,17,2007-01-26,4
2,3,2007-11-11,0
3,4,2007-12-05,1
11,12,2007-12-11,2
15,16,2007-12-20,3
39,40,2007-12-26,5
4,5,2008-01-06,1
47,48,2008-01-06,11
48,49,2008-01-07,12


In [485]:
# 175 Find the sales order ID, order date, and the rank of each order based on the total sales amount, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [486]:
soh['Rank'] = soh['TotalDue'].rank()

In [487]:
soh[['SalesOrderID' ,'OrderDate' ,'TotalDue', 'Rank']].sort_values('OrderDate').head()

,SalesOrderID,OrderDate,TotalDue,Rank
0,43660,2011-05-20,1457.3288,18895.0
24,43684,2011-05-31,6301.6258,29321.0
25,43685,2011-05-31,3082.0191,26928.0
26,43686,2011-05-31,3899.6756,27447.0
27,43687,2011-05-31,1416.9322,18834.0


In [488]:
# 176 Retrieve the customer ID, order date, and the percentage of total sales amount contributed by each order, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [489]:
total = soh['TotalDue'].sum()

In [490]:
soh['Contribution'] = (soh['TotalDue']/total)*100

In [491]:
soh[['SalesOrderID', 'CustomerID', 'OrderDate', 'TotalDue', 'Contribution']].head()

,SalesOrderID,CustomerID,OrderDate,TotalDue,Contribution
0,43660,29672,2011-05-20,1457.3288,0.001183
1,43661,29734,2011-05-31,36865.8012,0.029919
2,43662,29994,2011-05-31,32474.9324,0.026356
3,43663,29565,2011-05-31,472.3108,0.000383
4,43664,29898,2011-05-31,27510.4109,0.022327


In [492]:
# 177 List the product ID, name, and the difference between the current row's list price and the average list price of all products, ordered by product ID.

pro = db_reader().read_table('production.product', connection)
pro.head()


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [493]:
pro['avg'] = pro['ListPrice'].mean()

In [494]:
pro['diff'] = pro['ListPrice'] - pro['avg']

In [495]:
pro[['ProductID', 'Name', 'ListPrice', 'avg', 'diff']].head()

,ProductID,Name,ListPrice,avg,diff
0,1,Adjustable Race,1000.0,166.819324,833.180676
1,2,Bearing Ball,0.0,166.819324,-166.819324
2,3,BB Ball Bearing,0.0,166.819324,-166.819324
3,4,Headset Ball Bearings,0.0,166.819324,-166.819324
4,316,Blade,0.0,166.819324,-166.819324


In [496]:
# 178 Retrieve the employee ID, hire date, and the average number of employees hired per year, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [497]:
emp['Year'] = pd.to_datetime(emp['HireDate']).dt.year

In [498]:
count = emp.groupby('Year')['BusinessEntityID'].count().reset_index(name='count')
count

,Year,count
0,2006,1
1,2007,6
2,2008,75
3,2009,148
4,2010,38
5,2011,16
6,2012,4
7,2013,3


In [499]:
emp['avg'] = count['count'].mean()

In [500]:
emp[['BusinessEntityID', 'HireDate','Year', 'avg']].sort_values('HireDate').head()

,BusinessEntityID,HireDate,Year,avg
27,28,2006-06-30,2006,36.375
16,17,2007-01-26,2007,36.375
2,3,2007-11-11,2007,36.375
3,4,2007-12-05,2007,36.375
11,12,2007-12-11,2007,36.375


In [501]:
# 179 Find the sales order ID, order date, and the running total of sales amount for each year, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [502]:
soh['OrderDate'] = pd.to_datetime(soh['OrderDate'])

In [503]:
soh['Year']= soh['OrderDate'].dt.year

In [504]:
soh['RunningTotal'] = soh.groupby('Year')['TotalDue'].cumsum()

In [505]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue','Year' ,'RunningTotal']].tail()

,SalesOrderID,OrderDate,TotalDue,Year,RunningTotal
31460,75120,2014-06-30,93.8808,2014,2.241917e+07
31461,75121,2014-06-30,82.8529,2014,2.241925e+07
31462,75122,2014-06-30,34.2219,2014,2.241929e+07
31463,75123,2014-06-30,209.9169,2014,2.241950e+07
31464,75124,2011-05-31,23153.2339,2011,1.415570e+07


In [506]:
# 180 Retrieve the product ID, name, and the percentage of total revenue generated by each product, ordered by product ID.

pro = db_reader().read_table('production.product',connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [511]:
total = pro['ListPrice'].sum()

In [512]:
pro['Revenue'] = (pro['ListPrice']/total)*100

In [513]:
pro[['ProductID', 'Name', 'ListPrice', 'Revenue']].head()

,ProductID,Name,ListPrice,Revenue
0,1,Adjustable Race,1000.0,1.184686
1,2,Bearing Ball,0.0,0.000000
2,3,BB Ball Bearing,0.0,0.000000
3,4,Headset Ball Bearings,0.0,0.000000
4,316,Blade,0.0,0.000000


In [ ]:
# 181 List the employee ID, hire date, and the average tenure of employees hired in the same year, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

In [514]:
# 182 Retrieve the sales order ID, order date, and the difference in total sales amount from the previous order, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [517]:
soh['diff'] = soh['TotalDue'].diff()

In [518]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue', 'diff']].head()

,SalesOrderID,OrderDate,TotalDue,diff
0,43660,2011-05-20,1457.3288,NaN
1,43661,2011-05-31,36865.8012,35408.4724
2,43662,2011-05-31,32474.9324,-4390.8688
3,43663,2011-05-31,472.3108,-32002.6216
4,43664,2011-05-31,27510.4109,27038.1001


In [519]:
# 183 Find the product ID, name, and the rank of each product based on the total quantity sold, ordered by product ID.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [520]:
sod['Rank'] = sod['OrderQty'].rank()

In [522]:
sod[['ProductID', 'OrderQty', 'Rank']].sort_values('ProductID').head()

,ProductID,OrderQty,Rank
78722,707,1,37474.5
7619,707,1,37474.5
116446,707,1,37474.5
106352,707,1,37474.5
106361,707,1,37474.5


In [523]:
# 184 Retrieve the employee ID, hire date, and the number of employees hired in the same year as each employee, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [524]:
emp['Year'] = pd.to_datetime(emp['HireDate']).dt.year

In [525]:
count = emp.groupby('Year')['BusinessEntityID'].count().reset_index(name='count')
count

,Year,count
0,2006,1
1,2007,6
2,2008,75
3,2009,148
4,2010,38
5,2011,16
6,2012,4
7,2013,3


In [527]:
emp = merge().merge_tbl(emp[['BusinessEntityID', 'HireDate', 'Year']], count, 'inner', 'Year')
emp.head()

,BusinessEntityID,HireDate,Year,count
0,1,2009-01-14,2009,148
1,2,2008-01-31,2008,75
2,3,2007-11-11,2007,6
3,4,2007-12-05,2007,6
4,5,2008-01-06,2008,75


In [529]:
emp.sort_values('HireDate').head()

,BusinessEntityID,HireDate,Year,count
27,28,2006-06-30,2006,1
16,17,2007-01-26,2007,6
2,3,2007-11-11,2007,6
3,4,2007-12-05,2007,6
11,12,2007-12-11,2007,6


In [530]:
# 185 Find the sales order ID, order date, and the sum of total sales amount for the last 3 orders, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [533]:
soh['Running'] = soh['TotalDue'].rolling(3, min_periods=1).sum()

In [534]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue', 'Running']].head()

,SalesOrderID,OrderDate,TotalDue,Running
0,43660,2011-05-20,1457.3288,1457.3288
1,43661,2011-05-31,36865.8012,38323.1300
2,43662,2011-05-31,32474.9324,70798.0624
3,43663,2011-05-31,472.3108,69813.0444
4,43664,2011-05-31,27510.4109,60457.6541


In [535]:
# 186 Retrieve the product ID, name, and the average list price of products within the same subcategory, ordered by product ID.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [536]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [538]:
sub = merge().merge_tbl(pro[['ProductID', 'Name', 'ListPrice', 'ProductSubcategoryID']], sc[['ProductSubcategoryID', 'Name']], 'inner', 'ProductSubcategoryID')
sub.head()

,ProductID,Name_x,ListPrice,ProductSubcategoryID,Name_y
0,680,"HL Road Frame - Black, 58",1431.50,14.0,Road Frames
1,706,"HL Road Frame - Red, 58",1431.50,14.0,Road Frames
2,707,"Sport-100 Helmet, Red",34.99,31.0,Helmets
3,708,"Sport-100 Helmet, Black",34.99,31.0,Helmets
4,709,"Mountain Bike Socks, M",9.50,23.0,Socks


In [540]:
sc = sub.groupby('Name_y')['ListPrice'].mean().reset_index(name = 'avg')
sc.head()

,Name_y,avg
0,Bib-Shorts,89.99
1,Bike Racks,120.00
2,Bike Stands,159.00
3,Bottles and Cages,7.99
4,Bottom Brackets,92.24


In [541]:
sub = merge().merge_tbl(sub, sc, 'inner', 'Name_y')
sub.head()

,ProductID,Name_x,ListPrice,ProductSubcategoryID,Name_y,avg
0,680,"HL Road Frame - Black, 58",1431.50,14.0,Road Frames,780.043636
1,706,"HL Road Frame - Red, 58",1431.50,14.0,Road Frames,780.043636
2,707,"Sport-100 Helmet, Red",34.99,31.0,Helmets,34.990000
3,708,"Sport-100 Helmet, Black",34.99,31.0,Helmets,34.990000
4,709,"Mountain Bike Socks, M",9.50,23.0,Socks,9.176000


In [542]:
# 187 List the employee ID, hire date, and the difference in years between the hire date of each employee and the earliest hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [549]:
min = emp['HireDate'].dt.year.min()
min

2006

In [545]:
emp['HireDate'] = pd.to_datetime(emp['HireDate'])

In [550]:
emp['diff'] = emp['HireDate'].dt.year - min

In [551]:
emp[['BusinessEntityID', 'HireDate', 'diff']].head()

,BusinessEntityID,HireDate,diff
0,1,2009-01-14,3
1,2,2008-01-31,2
2,3,2007-11-11,1
3,4,2007-12-05,1
4,5,2008-01-06,2


In [552]:
# 188 Retrieve the sales order ID, order date, and the average of total sales amount for the last 5 orders, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [553]:
soh['Running'] = soh['TotalDue'].rolling(5, min_periods=1).sum()

In [554]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue', 'Running']].head()

,SalesOrderID,OrderDate,TotalDue,Running
0,43660,2011-05-20,1457.3288,1457.3288
1,43661,2011-05-31,36865.8012,38323.1300
2,43662,2011-05-31,32474.9324,70798.0624
3,43663,2011-05-31,472.3108,71270.3732
4,43664,2011-05-31,27510.4109,98780.7841


In [556]:
# 189 Find the product ID, name, and the difference in quantity sold from the previous order for each product, ordered by product ID.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [561]:
sod = sod[['ProductID', 'OrderQty']].sort_values('ProductID')
sod.head()

,ProductID,OrderQty
78722,707,1
7619,707,1
116446,707,1
106352,707,1
106361,707,1


In [562]:
sod['diff'] = sod['OrderQty'].diff()

In [563]:
sod.head(10)

,ProductID,OrderQty,diff
78722,707,1,NaN
7619,707,1,0.0
116446,707,1,0.0
106352,707,1,0.0
106361,707,1,0.0
48788,707,4,3.0
116431,707,1,-3.0
7605,707,1,0.0
21802,707,2,1.0
116413,707,1,-1.0


In [564]:
# 190 Retrieve the employee ID, hire date, and the running total of employee count for each hire year, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [571]:
emp['Year'] = pd.to_datetime(emp['HireDate']).dt.year

In [578]:
emp['RunningTotal'] = emp.groupby('Year')['BusinessEntityID'].cumcount()

In [579]:
emp[['BusinessEntityID', 'HireDate', 'RunningTotal']].sort_values('HireDate').head(10)

,BusinessEntityID,HireDate,RunningTotal
27,28,2006-06-30,0
16,17,2007-01-26,4
2,3,2007-11-11,0
3,4,2007-12-05,1
11,12,2007-12-11,2
15,16,2007-12-20,3
39,40,2007-12-26,5
4,5,2008-01-06,1
47,48,2008-01-06,11
48,49,2008-01-07,12


In [585]:
# 191 List the product ID, name, and the average list price of products in the same category, ordered by product ID.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [593]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()


,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [594]:
cat = merge().merge_tbl(pro[['ProductID', 'Name', 'ListPrice', 'ProductSubcategoryID']], sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,Name,ListPrice,ProductSubcategoryID,ProductCategoryID
0,680,"HL Road Frame - Black, 58",1431.50,14.0,2
1,706,"HL Road Frame - Red, 58",1431.50,14.0,2
2,707,"Sport-100 Helmet, Red",34.99,31.0,4
3,708,"Sport-100 Helmet, Black",34.99,31.0,4
4,709,"Mountain Bike Socks, M",9.50,23.0,3


In [595]:
c = cat.groupby('ProductCategoryID')['ListPrice'].mean().reset_index(name = 'avg')
c.head()

,ProductCategoryID,avg
0,1,146.674000
1,2,469.860299
2,3,49.822222
3,4,34.348966


In [596]:
cat = merge().merge_tbl(cat, c, 'inner', 'ProductCategoryID')
cat.head()

,ProductID,Name,ListPrice,ProductSubcategoryID,ProductCategoryID,avg
0,680,"HL Road Frame - Black, 58",1431.50,14.0,2,469.860299
1,706,"HL Road Frame - Red, 58",1431.50,14.0,2,469.860299
2,707,"Sport-100 Helmet, Red",34.99,31.0,4,34.348966
3,708,"Sport-100 Helmet, Black",34.99,31.0,4,34.348966
4,709,"Mountain Bike Socks, M",9.50,23.0,3,49.822222


In [597]:
# 192 Retrieve the product ID, name, and assign a unique row number to each product, ordered by product name.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [604]:
pro = pro[['ProductID', 'Name']].sort_values('Name')
pro.head()

,ProductID,Name
216,712,AWC Logo Cap
0,1,Adjustable Race
383,879,All-Purpose Bike Stand
2,3,BB Ball Bearing
505,2006,BIKE HANDLE


In [605]:
pro['Rank'] = pro['ProductID'].rank(method='dense')

In [606]:
pro.head()

,ProductID,Name,Rank
216,712,AWC Logo Cap,217.0
0,1,Adjustable Race,1.0
383,879,All-Purpose Bike Stand,384.0
2,3,BB Ball Bearing,3.0
505,2006,BIKE HANDLE,506.0


In [607]:
# 193 List the employee ID, hire date, and assign a rank to each employee based on their hire date, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [608]:
emp['rank'] = emp['HireDate'].rank()

In [610]:
emp[['BusinessEntityID', 'HireDate', 'rank']].sort_values('HireDate').head()

,BusinessEntityID,HireDate,rank
27,28,2006-06-30,1.0
16,17,2007-01-26,2.0
2,3,2007-11-11,3.0
3,4,2007-12-05,4.0
11,12,2007-12-11,5.0


In [611]:
# 194 Retrieve the customer ID, order date, and assign a dense rank to each customer based on their order date, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [616]:
soh = soh[['SalesOrderID', 'OrderDate']].sort_values('OrderDate')

In [617]:
soh['Rank'] = soh['OrderDate'].rank()

In [618]:
soh.head()

,SalesOrderID,OrderDate,Rank
0,43660,2011-05-20,1.0
24,43684,2011-05-31,22.5
25,43685,2011-05-31,22.5
26,43686,2011-05-31,22.5
27,43687,2011-05-31,22.5


In [619]:
# 195 Find the sales order ID, order date, and assign a unique row number to each order, partitioned by the sales person ID and ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [620]:
soh = soh[['SalesOrderID', 'OrderDate', 'SalesPersonID']]
soh.head()

,SalesOrderID,OrderDate,SalesPersonID
0,43660,2011-05-20,279.0
1,43661,2011-05-31,282.0
2,43662,2011-05-31,282.0
3,43663,2011-05-31,276.0
4,43664,2011-05-31,280.0


In [626]:
a = soh.groupby(['SalesPersonID', 'SalesOrderID'])['OrderDate'].max().reset_index().sort_values('OrderDate')
a.head()

,SalesPersonID,SalesOrderID,OrderDate
1623,279.0,43660,2011-05-20
2392,282.0,43674,2011-05-31
919,277.0,43691,2011-05-31
1627,279.0,43694,2011-05-31
1628,279.0,43695,2011-05-31


In [627]:
a['rank'] = a['SalesOrderID'].rank()

In [628]:
a.head()

,SalesPersonID,SalesOrderID,OrderDate,rank
1623,279.0,43660,2011-05-20,1.0
2392,282.0,43674,2011-05-31,15.0
919,277.0,43691,2011-05-31,32.0
1627,279.0,43694,2011-05-31,35.0
1628,279.0,43695,2011-05-31,36.0


In [646]:
# 196 Retrieve the product ID, name, and assign a rank to each product based on the list price, partitioned by the product category, ordered by product name

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [647]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [648]:
cat = merge().merge_tbl(pro[['ProductID', 'Name', 'ListPrice', 'ProductSubcategoryID']], sc[['ProductSubcategoryID', 'ProductCategoryID']],'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,Name,ListPrice,ProductSubcategoryID,ProductCategoryID
0,680,"HL Road Frame - Black, 58",1431.50,14.0,2
1,706,"HL Road Frame - Red, 58",1431.50,14.0,2
2,707,"Sport-100 Helmet, Red",34.99,31.0,4
3,708,"Sport-100 Helmet, Black",34.99,31.0,4
4,709,"Mountain Bike Socks, M",9.50,23.0,3


In [649]:
cat['rank'] = cat['ListPrice'].rank()

In [650]:
a = cat.groupby('ProductCategoryID')
a.head()

,ProductID,Name,ListPrice,ProductSubcategoryID,ProductCategoryID,rank
0,680,"HL Road Frame - Black, 58",1431.500,14.0,2,291.0
1,706,"HL Road Frame - Red, 58",1431.500,14.0,2,291.0
2,707,"Sport-100 Helmet, Red",34.990,31.0,4,35.5
3,708,"Sport-100 Helmet, Black",34.990,31.0,4,35.5
4,709,"Mountain Bike Socks, M",9.500,23.0,3,12.5
5,710,"Mountain Bike Socks, L",9.500,23.0,3,12.5
6,711,"Sport-100 Helmet, Blue",34.990,31.0,4,35.5
7,712,AWC Logo Cap,8.990,19.0,3,9.5
8,713,"Long-Sleeve Logo Jersey, S",49.990,21.0,3,52.5
9,714,"Long-Sleeve Logo Jersey, M",49.990,21.0,3,52.5


In [635]:
# 197 List the employee ID, hire date, and assign a dense rank to each employee based on the department ID, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [636]:
edh = db_reader().read_table('humanresources.employeedepartmenthistory', connection)
edh.head()

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13
1,2,1,1,2008-01-31,None,2008-01-30
2,3,1,1,2007-11-11,None,2007-11-10
3,4,1,1,2007-12-05,2010-05-30,2010-05-28
4,4,2,1,2010-05-31,None,2010-05-30


In [637]:
id = merge().merge_tbl(emp[['BusinessEntityID', 'HireDate']], edh[['BusinessEntityID', 'DepartmentID']], 'inner', 'BusinessEntityID')
id.head()

,BusinessEntityID,HireDate,DepartmentID
0,1,2009-01-14,16
1,2,2008-01-31,1
2,3,2007-11-11,1
3,4,2007-12-05,1
4,4,2007-12-05,2


In [638]:
id['dense'] = id['DepartmentID'].rank(method='dense')

In [639]:
id.head()

,BusinessEntityID,HireDate,DepartmentID,dense
0,1,2009-01-14,16,16.0
1,2,2008-01-31,1,1.0
2,3,2007-11-11,1,1.0
3,4,2007-12-05,1,1.0
4,4,2007-12-05,2,2.0


In [640]:
# 198 Retrieve the product ID, name, and assign a unique row number to each product, partitioned by the product subcategory, ordered by product ID.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [641]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [642]:
cat = merge().merge_tbl(pro[['ProductID', 'Name', 'ListPrice', 'ProductSubcategoryID']], sc[['ProductSubcategoryID']],'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,Name,ListPrice,ProductSubcategoryID
0,680,"HL Road Frame - Black, 58",1431.50,14.0
1,706,"HL Road Frame - Red, 58",1431.50,14.0
2,707,"Sport-100 Helmet, Red",34.99,31.0
3,708,"Sport-100 Helmet, Black",34.99,31.0
4,709,"Mountain Bike Socks, M",9.50,23.0


In [643]:
cat['rank'] = cat['ProductID'].rank()

In [644]:
cat.groupby('ProductSubcategoryID').head()

,ProductID,Name,ListPrice,ProductSubcategoryID,rank
0,680,"HL Road Frame - Black, 58",1431.500,14.0,1.0
1,706,"HL Road Frame - Red, 58",1431.500,14.0,2.0
2,707,"Sport-100 Helmet, Red",34.990,31.0,3.0
3,708,"Sport-100 Helmet, Black",34.990,31.0,4.0
4,709,"Mountain Bike Socks, M",9.500,23.0,5.0
5,710,"Mountain Bike Socks, L",9.500,23.0,6.0
6,711,"Sport-100 Helmet, Blue",34.990,31.0,7.0
7,712,AWC Logo Cap,8.990,19.0,8.0
8,713,"Long-Sleeve Logo Jersey, S",49.990,21.0,9.0
9,714,"Long-Sleeve Logo Jersey, M",49.990,21.0,10.0


In [651]:
# 199 Find the sales order ID, order date, and assign a rank to each order based on the total due amount, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [652]:
soh['Rank'] = soh['TotalDue'].rank()

In [654]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue', 'Rank']].head()

,SalesOrderID,OrderDate,TotalDue,Rank
0,43660,2011-05-20,1457.3288,18895.0
1,43661,2011-05-31,36865.8012,30449.0
2,43662,2011-05-31,32474.9324,30281.0
3,43663,2011-05-31,472.3108,12899.0
4,43664,2011-05-31,27510.4109,30086.0


In [655]:
# 200 Retrieve the product ID, name, and assign a dense rank to each product based on the weight, ordered by product ID.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [656]:
pro['Rank'] = pro['Weight'].rank()

In [658]:
pro[['ProductID', 'Name', 'Weight', 'Rank']].tail()

,ProductID,Name,Weight,Rank
501,997,"Road-750 Black, 44",19.77,115.0
502,998,"Road-750 Black, 48",20.13,122.0
503,999,"Road-750 Black, 52",20.42,125.0
504,1001,Vintage Handcrafted Wooden Handlebars,1.50,1.0
505,2006,BIKE HANDLE,2.00,2.0


In [659]:
# 201 List the employee ID, hire date, and assign a unique row number to each employee, partitioned by the job title, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [663]:
emp = emp[['BusinessEntityID', 'JobTitle', 'HireDate']]

In [668]:
emp['Rank'] = emp['BusinessEntityID'].rank()

In [667]:
emp.head()

,BusinessEntityID,JobTitle,HireDate,Rank
0,1,Chief Executive Officer,2009-01-14,<bound method NDFrame.rank of 0 1\n1 ...
1,2,Vice President of Engineering,2008-01-31,<bound method NDFrame.rank of 0 1\n1 ...
2,3,Engineering Manager,2007-11-11,<bound method NDFrame.rank of 0 1\n1 ...
3,4,Senior Tool Designer,2007-12-05,<bound method NDFrame.rank of 0 1\n1 ...
4,5,Design Engineer,2008-01-06,<bound method NDFrame.rank of 0 1\n1 ...


In [671]:
emp.groupby('JobTitle').head()

,BusinessEntityID,JobTitle,HireDate,Rank
0,1,Chief Executive Officer,2009-01-14,1.0
1,2,Vice President of Engineering,2008-01-31,2.0
2,3,Engineering Manager,2007-11-11,3.0
3,4,Senior Tool Designer,2007-12-05,4.0
4,5,Design Engineer,2008-01-06,5.0
5,6,Design Engineer,2008-01-24,6.0
6,7,Research and Development Manager,2009-02-08,7.0
7,8,Research and Development Engineer,2008-12-29,8.0
8,9,Research and Development Engineer,2009-01-16,9.0
9,10,Research and Development Manager,2009-05-03,10.0


In [672]:
# 202 Retrieve the sales order ID, order date, and the row number of each order within its customer, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [673]:
soh = soh[['SalesOrderID', 'OrderDate', 'CustomerID']]

In [674]:
soh['Rank'] = soh['CustomerID'].rank()

In [675]:
soh.sort_values('OrderDate').head()

,SalesOrderID,OrderDate,CustomerID,Rank
0,43660,2011-05-20,29672,28814.5
24,43684,2011-05-31,29912,30268.5
25,43685,2011-05-31,30084,31253.5
26,43686,2011-05-31,29606,28429.5
27,43687,2011-05-31,29610,28447.0


In [677]:
# 203 Find the product ID, name, and the dense rank of each product based on the total quantity sold, ordered by product ID.

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [679]:
sod['Rank'] = sod['OrderQty'].rank(method='dense')

In [681]:
sod[['ProductID', 'OrderQty', 'Rank']].sort_values('ProductID').head()

,ProductID,OrderQty,Rank
78722,707,1,1.0
7619,707,1,1.0
116446,707,1,1.0
106352,707,1,1.0
106361,707,1,1.0


In [682]:
# 204 Retrieve the employee ID, hire date, and the rank of each employee based on their hire year, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [684]:
emp['HireDate	'] = pd.to_datetime(emp['HireDate'])

In [685]:
emp['Year'] = emp['HireDate	'].dt.year

In [688]:
emp['Rank'] = emp['Year'].rank(method='dense')

In [689]:
emp[['BusinessEntityID', 'HireDate', 'Year', 'Rank']].sort_values('HireDate').head()

,BusinessEntityID,HireDate,Year,Rank
27,28,2006-06-30,2006,1.0
16,17,2007-01-26,2007,2.0
2,3,2007-11-11,2007,2.0
3,4,2007-12-05,2007,2.0
11,12,2007-12-11,2007,2.0


In [690]:
# 205 Find the product ID, name, and the row number of each product based on the list price, ordered by product ID.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [691]:
pro['Rank'] = pro['ListPrice'].rank(method='first')

In [692]:
pro[['ProductID', 'Name', 'ListPrice', 'Rank']].sort_values('ProductID').head()

,ProductID,Name,ListPrice,Rank
0,1,Adjustable Race,1000.0,476.0
1,2,Bearing Ball,0.0,1.0
2,3,BB Ball Bearing,0.0,2.0
3,4,Headset Ball Bearings,0.0,3.0
4,316,Blade,0.0,4.0


In [693]:
# 206 Retrieve the sales order ID, order date, and the rank of each order based on the total sales amount, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [694]:
soh['Rank'] = soh['TotalDue'].rank()

In [695]:
soh[['SalesOrderID', 'OrderDate', 'TotalDue', 'Rank']].sort_values('OrderDate').head()

,SalesOrderID,OrderDate,TotalDue,Rank
0,43660,2011-05-20,1457.3288,18895.0
24,43684,2011-05-31,6301.6258,29321.0
25,43685,2011-05-31,3082.0191,26928.0
26,43686,2011-05-31,3899.6756,27447.0
27,43687,2011-05-31,1416.9322,18834.0


In [696]:
# 207 List the employee ID, hire date, and the dense rank of each employee based on the years of experience, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [697]:
emp['HireDate'] = pd.to_datetime(emp['HireDate'])

In [698]:
emp['exp'] = pd.Timestamp.now().year - emp['HireDate'].dt.year

In [699]:
emp['Rank'] = emp['exp'].rank(method='dense')

In [700]:
emp[['BusinessEntityID', 'HireDate', 'exp', 'Rank']].sort_values('HireDate').head()

,BusinessEntityID,HireDate,exp,Rank
27,28,2006-06-30,18,8.0
16,17,2007-01-26,17,7.0
2,3,2007-11-11,17,7.0
3,4,2007-12-05,17,7.0
11,12,2007-12-11,17,7.0


In [701]:
# 208 Retrieve the product ID, name, and the row number of each product within its category, ordered by product ID.

pro = db_reader().read_table('production.product', connection)
pro.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [702]:
sc = db_reader().read_table('production.productsubcategory', connection)
sc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [703]:
cat = merge().merge_tbl(pro[['ProductID','Name','ProductSubcategoryID']], sc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
cat.head()

,ProductID,Name,ProductSubcategoryID,ProductCategoryID
0,680,"HL Road Frame - Black, 58",14.0,2
1,706,"HL Road Frame - Red, 58",14.0,2
2,707,"Sport-100 Helmet, Red",31.0,4
3,708,"Sport-100 Helmet, Black",31.0,4
4,709,"Mountain Bike Socks, M",23.0,3


In [704]:
cat['Rank'] = cat.groupby('ProductCategoryID')['ProductID'].rank(method='first')

In [706]:
cat[['ProductID', 'Name', 'ProductCategoryID', 'Rank']].sort_values('ProductID').head()

,ProductID,Name,ProductCategoryID,Rank
0,680,"HL Road Frame - Black, 58",2,1.0
1,706,"HL Road Frame - Red, 58",2,2.0
2,707,"Sport-100 Helmet, Red",4,1.0
3,708,"Sport-100 Helmet, Black",4,2.0
4,709,"Mountain Bike Socks, M",3,1.0


In [707]:
# 209 Find the sales order ID, order date, and the rank of each order based on the order year, ordered by order date

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [708]:
soh['OrderDate'] = pd.to_datetime(soh['OrderDate'])

In [709]:
soh['Year'] = soh['OrderDate'].dt.year

In [710]:
soh['Rank'] = soh['Year'].rank()

In [712]:
soh[['SalesOrderID', 'OrderDate', 'Year', 'Rank']].sort_values('OrderDate').head()

,SalesOrderID,OrderDate,Year,Rank
0,43660,2011-05-20,2011,804.0
24,43684,2011-05-31,2011,804.0
25,43685,2011-05-31,2011,804.0
26,43686,2011-05-31,2011,804.0
27,43687,2011-05-31,2011,804.0


In [713]:
# 210 Retrieve the customer ID, order date, and the dense rank of each customer based on the total sales amount, ordered by order date.

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [714]:
soh['Rank'] = soh['TotalDue'].rank(method='dense')

In [715]:
soh[['CustomerID', 'OrderDate', 'TotalDue','Rank']].sort_values('OrderDate').head()

,CustomerID,OrderDate,TotalDue,Rank
0,29672,2011-05-20,1457.3288,1461.0
24,29912,2011-05-31,6301.6258,2630.0
25,30084,2011-05-31,3082.0191,2238.0
26,29606,2011-05-31,3899.6756,2344.0
27,29610,2011-05-31,1416.9322,1435.0


In [716]:
# 211 List the employee ID, hire date, and the row number of each employee within its department, ordered by hire date.

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [718]:
edh = db_reader().read_table('humanresources.employeedepartmenthistory', connection)
edh.head()

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13
1,2,1,1,2008-01-31,None,2008-01-30
2,3,1,1,2007-11-11,None,2007-11-10
3,4,1,1,2007-12-05,2010-05-30,2010-05-28
4,4,2,1,2010-05-31,None,2010-05-30


In [719]:
id = merge().merge_tbl(emp[['BusinessEntityID', 'HireDate']], edh[['BusinessEntityID', 'DepartmentID']], 'inner', 'BusinessEntityID')
id.head() 

,BusinessEntityID,HireDate,DepartmentID
0,1,2009-01-14,16
1,2,2008-01-31,1
2,3,2007-11-11,1
3,4,2007-12-05,1
4,4,2007-12-05,2


In [720]:
id['rank'] = id.groupby('DepartmentID')['BusinessEntityID'].rank(method='first')

In [721]:
id.sort_values('HireDate').head()

,BusinessEntityID,HireDate,DepartmentID,rank
29,28,2006-06-30,7,3.0
18,17,2007-01-26,4,2.0
2,3,2007-11-11,1,2.0
3,4,2007-12-05,1,3.0
4,4,2007-12-05,2,1.0
